In [ ]:
import pandas as pd
electoral_result = pd.read_csv('/workspaces/My_Vault/electoral_analysis/electoral_result_data.csv')

print(electoral_result.head())


     state  year  ... candidatevotes totalvotes
0  ALABAMA  1976  ...         659170    1182850
1  ALABAMA  1976  ...         504070    1182850
2  ALABAMA  1976  ...           9198    1182850
3  ALABAMA  1976  ...           6669    1182850
4  ALABAMA  1976  ...           1954    1182850

[5 rows x 7 columns]
